In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive





# 학습된 Fasttext 적용 해서 X_train, y_train, X_test 만들기




In [83]:
import pandas as pd
train_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청/train_preprocesed.csv")
test_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청/test_preprocessed.csv")

In [84]:
!pip install jamo

In [85]:
from jamo import h2j, j2hcj
#자모단위 자르는 함수
def word2jamo(sen):
    return j2hcj(h2j(sen))
train_preprocessed['jamo'] = train_preprocessed['document'].apply(word2jamo)
test_preprocessed['jamo'] = test_preprocessed['document'].apply(word2jamo)

In [87]:
import fasttext

fasttext_embedding_학습_ver0.1에서 학습한 fasttext 임베딩 모델 불러오기

In [88]:
model = fasttext.load_model('/content/MyDrive/MyDrive/통계청/embeddingmodels/fasttext_ver0.1.bin')

In [89]:
vec_list_train = []
for i in train_preprocessed['jamo']:
    vec = model.get_sentence_vector(i)
    vec_list_train.append(vec)

vec_list_test = []
for i in test_preprocessed['jamo']:
    vec = model.get_sentence_vector(i)
    vec_list_test.append(vec)

SyntaxError: ignored

label 개수가 230개미만인것 없애기

In [ ]:
train_preprocessed['emb_vec'] = vec_list_train
test_preprocessed['emb_vec'] = vec_list_test

In [ ]:
countdf = train_preprocessed.groupby('label').count().sort_values(by='document')
over230labels = countdf[countdf['document']>230].index.to_list()
train_preprocessed = train_preprocessed[train_preprocessed['label'].isin(over230labels)]

train_processed 에서 임베딩벡터와 label을 각각 X_train,y_train 로 뽑아내서 저장, test_preprecessed의 임베딩벡터를 X_test로 저장

In [ ]:
X_train = np.array(train_preprocessed['emb_vec'].to_list())
y_train = np.array(train_preprocessed['label'].to_list())
X_test = np.array(test_preprocessed['emb_vec'].to_list())


In [ ]:
X_train.shape

In [ ]:
np.save('/content/MyDrive/MyDrive/통계청/embedded_numpy/X_train_ver0.1.npy', X_train)

In [ ]:
np.save('/content/MyDrive/MyDrive/통계청/embedded_numpy/X_test_ver0.1.npy', X_test)

In [ ]:
np.save('/content/MyDrive/MyDrive/통계청/embedded_numpy/y_train_ver0.1.npy', y_train)

# 딥러닝 모델 구축 시작

In [42]:
import numpy as np
X_train = np.load('/content/MyDrive/MyDrive/통계청/embedded_numpy/X_train_ver0.1.npy',allow_pickle=True)
y_train = np.load('/content/MyDrive/MyDrive/통계청/embedded_numpy/y_train_ver0.1.npy',allow_pickle=True)

In [43]:
import tensorflow as tf
import pandas as pd

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping


In [44]:
x_train.astype(float)

array([[-0.06599911,  0.05893984, -0.01424733, ...,  0.09704771,
         0.07881667,  0.00334301],
       [ 0.00033214,  0.08497984, -0.07324332, ...,  0.05424456,
        -0.02190229, -0.03212171],
       [ 0.03648109, -0.03468831, -0.06038803, ...,  0.07813241,
         0.00637706,  0.0178444 ],
       ...,
       [ 0.06898117,  0.02758568,  0.03475236, ...,  0.07755586,
        -0.02388088,  0.02220103],
       [ 0.0861293 ,  0.05143277,  0.16811433, ...,  0.15496682,
         0.06009046,  0.00797901],
       [ 0.10387035,  0.04931091, -0.03208455, ...,  0.00954804,
         0.06546913, -0.03701732]])

In [45]:
y_train

array(['S95952', 'G47472', 'G46467', ..., 'G47474', 'P85856', 'I56561'],
      dtype='<U6')

X_train과 y_train을 trainset,validation set으로 분할

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encoder.fit(y_train)
y_train_digit = encoder.transform(y_train)
x_train, x_val, y_train, y_val = train_test_split(X_train,y_train_digit, 
                                                    test_size=0.2,  
                                                    random_state=1004)



In [32]:
X_train.shape

(995442, 100)

In [31]:
y_train.shape

(796353,)

In [70]:

#from tensorflow.keras.utils import plot_model

model1 = Sequential(name = 'model1')
model1.add(Dense(100, activation='relu', input_dim=(100)))
model1.add(Dense(1024, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1024, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(168, activation='softmax'))
model1.summary()
#plot_model(model, show_shapes=True)

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 100)               10100     
                                                                 
 dense_39 (Dense)            (None, 1024)              103424    
                                                                 
 dropout_16 (Dropout)        (None, 1024)              0         
                                                                 
 dense_40 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_17 (Dropout)        (None, 1024)              0         
                                                                 
 dense_41 (Dense)            (None, 168)               172200    
                                                                 
Total params: 1,335,324
Trainable params: 1,335,324
Non-trai

In [75]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [73]:
from tensorflow import keras

In [76]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,restore_best_weights=True)
history = model1.fit(x_train,y_train,
                    epochs=100,
                    batch_size=100,
                    validation_data=(x_val,y_val),
                    callbacks=early_stop)

Epoch 1/100
7964/7964 [==============================] - 29s 3ms/step - loss: 0.5350 - accuracy: 0.8538 - val_loss: 0.3907 - val_accuracy: 0.8902
Epoch 2/100
7964/7964 [==============================] - 28s 3ms/step - loss: 0.3848 - accuracy: 0.8917 - val_loss: 0.3561 - val_accuracy: 0.9009
Epoch 3/100
7964/7964 [==============================] - 26s 3ms/step - loss: 0.3545 - accuracy: 0.8997 - val_loss: 0.3442 - val_accuracy: 0.9038
Epoch 4/100
7964/7964 [==============================] - 26s 3ms/step - loss: 0.3376 - accuracy: 0.9038 - val_loss: 0.3421 - val_accuracy: 0.9045
Epoch 5/100
7964/7964 [==============================] - 35s 4ms/step - loss: 0.3253 - accuracy: 0.9067 - val_loss: 0.3335 - val_accuracy: 0.9068
Epoch 6/100
7964/7964 [==============================] - 35s 4ms/step - loss: 0.3163 - accuracy: 0.9090 - val_loss: 0.3364 - val_accuracy: 0.9064
Epoch 7/100
7964/7964 [==============================] - 28s 3ms/step - loss: 0.3092 - accuracy: 0.9109 - val_loss: 0.3373 -

In [77]:
model2 = Sequential(name = 'model1')
model2.add(Dense(100, activation='relu', input_dim=(100)))
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(168, activation='softmax'))
model2.summary()

model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,restore_best_weights=True)
history_model2 = model2.fit(x_train,y_train,
                    epochs=100,
                    batch_size=100,
                    validation_data=(x_val,y_val),
                    callbacks=early_stop)

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 100)               10100     
                                                                 
 dense_43 (Dense)            (None, 512)               51712     
                                                                 
 dropout_18 (Dropout)        (None, 512)               0         
                                                                 
 dense_44 (Dense)            (None, 512)               262656    
                                                                 
 dropout_19 (Dropout)        (None, 512)               0         
                                                                 
 dense_45 (Dense)            (None, 168)               86184     
                                                                 
Total params: 410,652
Trainable params: 410,652
Non-trainabl

In [78]:
model3 = Sequential(name = 'model3')
model3.add(Dense(100, activation='relu', input_dim=(100)))
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(168, activation='softmax'))
model3.summary()

model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,restore_best_weights=True)
history_model3 = model3.fit(x_train,y_train,
                    epochs=100,
                    batch_size=100,
                    validation_data=(x_val,y_val),
                    callbacks=early_stop)

Model: "model3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 100)               10100     
                                                                 
 dense_47 (Dense)            (None, 256)               25856     
                                                                 
 dropout_20 (Dropout)        (None, 256)               0         
                                                                 
 dense_48 (Dense)            (None, 256)               65792     
                                                                 
 dropout_21 (Dropout)        (None, 256)               0         
                                                                 
 dense_49 (Dense)            (None, 256)               65792     
                                                                 
 dropout_22 (Dropout)        (None, 256)               0    

In [82]:

model5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4,restore_best_weights=True)
history_model5 = model5.fit(x_train,y_train,
                    epochs=100,
                    batch_size=100,
                    validation_data=(x_val,y_val),
                    callbacks=early_stop)

Epoch 1/100
7964/7964 [==============================] - 28s 4ms/step - loss: 0.5236 - accuracy: 0.8567 - val_loss: 0.3933 - val_accuracy: 0.8896
Epoch 2/100
7964/7964 [==============================] - 29s 4ms/step - loss: 0.3803 - accuracy: 0.8934 - val_loss: 0.3541 - val_accuracy: 0.9012
Epoch 3/100
7964/7964 [==============================] - 29s 4ms/step - loss: 0.3509 - accuracy: 0.9009 - val_loss: 0.3470 - val_accuracy: 0.9028
Epoch 4/100
7964/7964 [==============================] - 29s 4ms/step - loss: 0.3339 - accuracy: 0.9049 - val_loss: 0.3404 - val_accuracy: 0.9043
Epoch 5/100
7964/7964 [==============================] - 29s 4ms/step - loss: 0.3215 - accuracy: 0.9079 - val_loss: 0.3415 - val_accuracy: 0.9056
Epoch 6/100
7964/7964 [==============================] - 29s 4ms/step - loss: 0.3129 - accuracy: 0.9101 - val_loss: 0.3373 - val_accuracy: 0.9073
Epoch 7/100
7964/7964 [==============================] - 30s 4ms/step - loss: 0.3059 - accuracy: 0.9117 - val_loss: 0.3371 -